In [1]:
from pyamaze import maze,COLOR,agent
import numpy as np
import time

In [2]:
col_value=20
row_vaule=20
test_maze=maze(rows=row_vaule,cols=col_value)
test_maze.CreateMaze(loopPercent=10)
# test_maze.CreateMaze(loadMaze='maze--mul_20_20.csv')
test_maze_mapping=test_maze.maze_map

In [3]:
# Retruns a 2d Array With 1 representing path and 
# Retruns a 2d Array With 1 representing path and 
def get_Matrix_mapping(test_maze, start_cell,goal_cell):
    key_map=dict()
    row=test_maze.rows
    col=test_maze.cols
    maze_mapping = test_maze.maze_map
    maze_2d=np.full(((2*(row))-1,(2*(col))-1),-1)
    maze_2d=np.pad(maze_2d, 1, mode='constant',constant_values= 1)
    row_val_matrix=maze_2d.shape[0]
    col_val_matrix=maze_2d.shape[1]
    ## Filling Boarders with Walls 
    for r in range(row_val_matrix):
        if(r % 2 !=0):
            continue
        for c in range(col_val_matrix):
                maze_2d[r][c]=1
    for c in range(col_val_matrix):
        if(c % 2 !=0):
            continue
        for r in range(row_val_matrix):
            maze_2d[r][c]=1
    for row_map in range(row):
        old_row=row_map+1
        new_row=old_row+row_map
        for col_map in range (col):
            old_col=col_map+1
            new_col=old_col+col_map
            maze_2d[new_row][new_col]=0
            new_points=(new_row,new_col)
            old_points=(old_row,old_col)
            key_map[new_points]=old_points
            if(maze_mapping[(old_row,old_col)]["W"]==1):
                maze_2d[new_row][(new_col-1)]=0
            else:
                maze_2d[new_row][(new_col-1)]=1 
            if(maze_mapping[(old_row,old_col)]["E"]==1):
                maze_2d[new_row][(new_col+1)]=0
            else:
                maze_2d[new_row][(new_col+1)]=1 
            if(maze_mapping[(old_row,old_col)]["N"]==1):
                maze_2d[(new_row-1)][(new_col)]=0
            else:
                maze_2d[(new_row-1)][(new_col)]=1 
            if(maze_mapping[(old_row,old_col)]["S"]==1):
                maze_2d[(new_row+1)][(new_col)]=0
            else:
                maze_2d[(new_row+1)][(new_col)]=1
            maze_2d[goal_cell[0]][goal_cell[1]]=2
    return maze_2d,key_map

    

In [4]:
def map_optimal_path_maze(optimal_path,maze_matrix_key_mapping):
    maze_path_val=list()
    for ele in optimal_path:
        if(ele in maze_matrix_key_mapping):
            maze_path_val.append(k[ele])
    final_path=dict()
    for i in range(1,len(maze_path_val)):
        final_path[maze_path_val[i]]=maze_path_val[i-1]
    return final_path

In [5]:

goal_cell=((2*(test_maze.rows))-1),((2*(test_maze.cols))-1)
start_cell=(1,1)

test_maze_2d, k=get_Matrix_mapping(test_maze,start_cell,goal_cell)
end_reward = 100
step_reward = -1
discount_factor = 0.99
max_iterations = 10000
actions = {
    "UP": (-1, 0),
    "DOWN": (1, 0),
    "LEFT": (0, -1),
    "RIGHT": (0, 1)
}

# State Matrix is defined and initial state is assigned to 0
states_matrix = [(r, c) for r in range(test_maze_2d.shape[0]) for c in range(test_maze_2d.shape[1])]
state_value_function = {state: 0 for state in states_matrix}

## Assign Policy and  Transition function

policy_matrix = {state: np.random.choice(list(actions.keys())) for state in states_matrix}
transition_function = {
    state: {
        action: {
            next_state: 1 if test_maze_2d[next_state] != 1 else 0
            for next_state in [tuple(np.array(state) + np.array(actions[action]))]
            if (0 <= next_state[0] < test_maze_2d.shape[0]) and (0 <= next_state[1] < test_maze_2d.shape[1])
        }
        for action in actions.keys()
    }
    for state in states_matrix
}
start_time = time.time()
## Performing Policy Iteration
counter=0
for i in range(max_iterations):
    # evaluate the current policy
    delta = 0
    for state in states_matrix:
        counter=counter+1
        old_state_value = state_value_function[state]
        action = policy_matrix[state]
        state_value_function[state] = sum(
            transition_function[state][action][next_state] * (
                (end_reward if test_maze_2d[next_state] == 2 else 0) +
                (step_reward if test_maze_2d[next_state] == 0 else 0) +
                discount_factor * state_value_function[next_state]
            )
            for next_state in transition_function[state][action].keys()
        )
        delta = max(delta, abs(old_state_value - state_value_function[state]))

    # check if the policy is optimal
    if delta < 1e-6:
        print("Total_No_Iterations",i)
        print("State",counter)
        break

    # improve the policy
    for state in states_matrix:
        best_action = None
        best_action_value = float("-inf")
        for action in actions.keys():
            action_value = sum(
                transition_function[state][action][next_state] * (
                    (end_reward if test_maze_2d[next_state] == 2 else 0) +
                    (step_reward if test_maze_2d[next_state] == 0 else 0) +
                    discount_factor * state_value_function[next_state]
                )
                for next_state in transition_function[state][action].keys()
            )
            if action_value > best_action_value:
                best_action = action
                best_action_value = action_value
        policy_matrix[state] = best_action

# print(policy_matrix)
# define the initial state and the list of actions
current_state = (1, 1)
actions_taken = []

# follow the policy until the goal state is reached
while test_maze_2d[current_state] != 2:
    # add the current state to the list of actions
    actions_taken.append(current_state)

    # take the action prescribed by the policy
    action = policy_matrix[current_state]
    next_state = tuple(np.array(current_state) + np.array(actions[action]))
    current_state = next_state

# add the goal state to the list of actions
actions_taken.append(current_state)

# print the list of actions
# print("Optimal path:")
# for state in actions_taken:
#     print(state)
exec_time = (time.time() - start_time)
print("--- Running Time : %s seconds ---" % exec_time)
pt=map_optimal_path_maze(actions_taken,k)
len(pt)
a=agent(test_maze,footprints=True,color=COLOR.yellow,shape='arrow',filled=True)
test_maze.tracePath({a:pt},delay=100)
test_maze.run()

Total_No_Iterations 956
State 1608717
--- Running Time : 12.677089929580688 seconds ---


2023-03-11 18:51:04.617 python[3932:191016] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (12.79 secs).
